## Esri GIS Tech 2022

### Notebook Webhook

#### Run this cell to connect to your GIS and get started:

In [ ]:
from arcgis.gis import GIS
gis = GIS("home")

#### Get the Template Groups Content

In [ ]:
templateGroup = gis.groups.get("670b326fc4bd44e0970c5a84c8c61091")

templateContent = templateGroup.content()
print(f"Found {len(templateContent)} Items in Template Group")

#### Get the latest WebHook Payload to retrieve GroupID

In [ ]:
webhookManager = gis.admin.webhooks
createGroupWebhook = [webhook for webhook in webhookManager.list() if "/groups/add" in webhook.properties["events"]][0]
print(f"Found Webhook {createGroupWebhook.properties['name']}")

In [ ]:
import json
webhookNotifications = createGroupWebhook.notifications
if len(webhookNotifications) > 0:
    latestEvent = createGroupWebhook.notifications[0]
    createGroupEvents = [event for event in json.loads(latestEvent["payload"])["events"] if event["operation"] == "add" and event["source"] == "group"]
    newGroups = [gis.groups.get(event["id"]) for event in createGroupEvents]
    print(f"Found {len(newGroups)} newly created group(s)")
else:
    raise SystemExit("No newly created groups found")

#### Clone the Content and share to the new Group

In [ ]:
for newGroup in newGroups:
    clonedItems = gis.content.clone_items(templateContent, search_existing_items=False)
    sharedItems = [item.share(groups=[newGroup]) for item in clonedItems]
    print(f"Succesfully shared {len([item for item in sharedItems if item['results'][0]['success']])} Items with group: {newGroup.title}")

#### Add Tags for User management and reassign to Administrator

In [ ]:
updateTagsSuccess = [newGroup.update(tags=["ArcGISUserRole_iBBBBBBBBBBBBBBB", "ArcGISUserType_creatorUT", "activedirectory_Software ontwikkeling"]) for newGroup in newGroups]
adminUsername = "giem_dev"
reassignSuccess = newGroup.reassign_to(adminUsername)
if updateTagsSuccess and reassignSuccess:
    print(f"Successfully added Tags for user management and reassigned group to: {adminUsername}")